In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from pprint import pprint
import glob

from pymedext_eds.annotators import Endlines, SentenceTokenizer, Hypothesis, ATCDFamille, SyntagmeTokenizer, Negation, RegexMatcher, rawtext_loader

In [13]:
file_list = glob.glob('/export/home/cse180025/tmp_fast/for_inference_covid_v2/2980/*.txt')
chunk = [rawtext_loader(x) for x in file_list]

In [9]:
endlines = Endlines(['raw_text'], 'endlines', 'endlines:v1')
sentences = SentenceTokenizer(['endlines'], 'sentence', 'sentenceTokenizer:v1')
hypothesis = Hypothesis(['sentence'], 'hypothesis', 'hypothesis:v1')
family = ATCDFamille(['sentence'], 'context', 'ATCDfamily:v1')
syntagmes = SyntagmeTokenizer(['sentence'], 'syntagme', 'SyntagmeTokenizer:v1')
negation = Negation(['syntagme'], 'negation', 'Negation:v1')
regex = RegexMatcher(['endlines','syntagme'], 'regex', 'RegexMatcher:v1', 'list_regexp.json')

In [10]:
import time
t = time.time()
for c in chunk[:30]: 
    c.annotate([endlines, sentences, hypothesis, family, syntagmes, negation, regex])
print(time.time()-t)

0.9896223545074463
